In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from sklearn import datasets
import scipy.stats as stats

In [2]:
# Read the csv file
crime_data = pd.read_csv("Crime_Reports.csv")

C:\Users\nancy\AppData\Local\Temp\ipykernel_22552\2761703141.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_data = pd.read_csv("Crime_Reports.csv")


In [3]:
crime_data.head()

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,2.006471e+09,FAMILY DISTURBANCE,3400,N,2/16/2006 14:25,2/16/2006,1425.0,2/16/2006 14:25,2/16/2006,1425.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.017330e+10,FAMILY DISTURBANCE,3400,N,11/26/2017 7:43,11/26/2017,743.0,11/26/2017 7:43,11/26/2017,743.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.004504e+10,TAMPERING WITH ID NUMBER,2719,N,9/14/2004 15:32,9/14/2004,1532.0,9/14/2004 15:32,9/14/2004,1532.0,...,NaN,N,9/14/2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.006961e+09,FAMILY DISTURBANCE,3400,N,4/6/2006 10:29,4/6/2006,1029.0,4/6/2006 10:29,4/6/2006,1029.0,...,NaN,N,5/1/2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.013851e+09,SEXUAL ASSAULT OF CHILD/OBJECT,1707,Y,1/1/2009 0:01,1/1/2009,1.0,3/26/2013 16:56,3/26/2013,1656.0,...,4.0,C,4/11/2013,11C,Rape,0.0,0.0,NaN,NaN,NaN


In [4]:
crime_data.columns

Index(['Incident Number', 'Highest Offense Description',
       'Highest Offense Code', 'Family Violence', 'Occurred Date Time',
       'Occurred Date', 'Occurred Time', 'Report Date Time', 'Report Date',
       'Report Time', 'Location Type', 'Address', 'Zip Code',
       'Council District', 'APD Sector', 'APD District', 'PRA', 'Census Tract',
       'Clearance Status', 'Clearance Date', 'UCR Category',
       'Category Description', 'X-coordinate', 'Y-coordinate', 'Latitude',
       'Longitude', 'Location'],
      dtype='object')

In [5]:
reduced_data = crime_data[["Highest Offense Description", "Family Violence", "Occurred Date","Location Type",
                           "Zip Code", "Latitude", "Longitude"]]
reduced_data

,Highest Offense Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude
0,FAMILY DISTURBANCE,N,2/16/2006,RESIDENCE / HOME,NaN,NaN,NaN
1,FAMILY DISTURBANCE,N,11/26/2017,RESIDENCE / HOME,NaN,NaN,NaN
2,TAMPERING WITH ID NUMBER,N,9/14/2004,NaN,NaN,NaN,NaN
3,FAMILY DISTURBANCE,N,4/6/2006,RESIDENCE / HOME,NaN,NaN,NaN
4,SEXUAL ASSAULT OF CHILD/OBJECT,Y,1/1/2009,RESIDENCE / HOME,78705.0,NaN,NaN
...,...,...,...,...,...,...,...
1048570,THEFT,N,6/1/2022,DEPARTMENT / DISCOUNT STORE,78753.0,30.339152,-97.691166
1048571,CRED CARD ABUSE - OTHER,N,1/2/2013,RESIDENCE / HOME,78748.0,30.154182,-97.815455
1048572,BURGLARY OF VEHICLE,N,11/30/2011,PARKING /DROP LOT/ GARAGE,78758.0,30.400556,-97.705303
1048573,FAMILY DISTURBANCE,N,7/19/2011,RESIDENCE / HOME,78758.0,30.367329,-97.704502


In [6]:
reduced_df = reduced_data.dropna()
reduced_df

,Highest Offense Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude
4463,ASSAULT BY THREAT,N,6/21/2023,RESIDENCE / HOME,78757.0,30.352698,-97.718252
6410,AUTO THEFT,N,6/5/2023,PARKING /DROP LOT/ GARAGE,78745.0,30.219033,-97.767189
7578,FAMILY DISTURBANCE,N,5/29/2023,RESIDENCE / HOME,78747.0,30.129830,-97.776806
28034,FAMILY DISTURBANCE,N,6/14/2023,RESIDENCE / HOME,78754.0,30.382122,-97.650877
28713,THEFT OF LICENSE PLATE,N,6/14/2023,CONSTRUCTION SITE,78747.0,30.125041,-97.790306
...,...,...,...,...,...,...,...
1048570,THEFT,N,6/1/2022,DEPARTMENT / DISCOUNT STORE,78753.0,30.339152,-97.691166
1048571,CRED CARD ABUSE - OTHER,N,1/2/2013,RESIDENCE / HOME,78748.0,30.154182,-97.815455
1048572,BURGLARY OF VEHICLE,N,11/30/2011,PARKING /DROP LOT/ GARAGE,78758.0,30.400556,-97.705303
1048573,FAMILY DISTURBANCE,N,7/19/2011,RESIDENCE / HOME,78758.0,30.367329,-97.704502


In [7]:
reduced_df["Occurred Date"] = pd.to_datetime(reduced_df["Occurred Date"])

C:\Users\nancy\AppData\Local\Temp\ipykernel_22552\3326495305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Occurred Date"] = pd.to_datetime(reduced_df["Occurred Date"])


In [8]:
reduced_df["Zip Code"] = reduced_df["Zip Code"].astype('int64')


C:\Users\nancy\AppData\Local\Temp\ipykernel_22552\111503900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df["Zip Code"] = reduced_df["Zip Code"].astype('int64')


In [9]:
reduced_df.dtypes

Highest Offense Description            object
Family Violence                        object
Occurred Date                  datetime64[ns]
Location Type                          object
Zip Code                                int64
Latitude                              float64
Longitude                             float64
dtype: object

In [10]:
reduced_df.head()

,Highest Offense Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude
4463,ASSAULT BY THREAT,N,2023-06-21,RESIDENCE / HOME,78757,30.352698,-97.718252
6410,AUTO THEFT,N,2023-06-05,PARKING /DROP LOT/ GARAGE,78745,30.219033,-97.767189
7578,FAMILY DISTURBANCE,N,2023-05-29,RESIDENCE / HOME,78747,30.129830,-97.776806
28034,FAMILY DISTURBANCE,N,2023-06-14,RESIDENCE / HOME,78754,30.382122,-97.650877
28713,THEFT OF LICENSE PLATE,N,2023-06-14,CONSTRUCTION SITE,78747,30.125041,-97.790306


In [11]:
covid_crime = reduced_df[(reduced_df["Occurred Date"] > "2019-01-01") & (reduced_df["Occurred Date"] < "2022-01-01")]
covid_crime.head()

,Highest Offense Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude
34745,AUTO THEFT,N,2019-09-27,PARKING /DROP LOT/ GARAGE,78751,30.306088,-97.715062
34761,IDENTITY THEFT,N,2021-09-08,RESIDENCE / HOME,78758,30.403869,-97.698004
34869,BURGLARY OF RESIDENCE,Y,2021-12-09,RESIDENCE / HOME,78753,30.352253,-97.692818
34995,FAMILY DISTURBANCE,N,2021-06-20,HWY / ROAD / ALLEY/ STREET/ SIDEWALK,78745,30.198187,-97.769155
35160,AGG ASSAULT FAM/DATE VIOLENCE,Y,2019-08-03,RESIDENCE / HOME,78754,30.360525,-97.638261


In [12]:
crimes_df = covid_crime.groupby("Highest Offense Description").count().sort_values(by = "Family Violence",ascending = False)
crimes_df2 = crimes_df.rename(columns ={"Family Violence" : "Crime Count"})
crimes_df2.head()

,Crime Count,Occurred Date,Location Type,Zip Code,Latitude,Longitude
Highest Offense Description,,,,,,
FAMILY DISTURBANCE,12449,12449,12449,12449,12449,12449
BURGLARY OF VEHICLE,10667,10667,10667,10667,10667,10667
THEFT,9395,9395,9395,9395,9395,9395
CRIMINAL MISCHIEF,6596,6596,6596,6596,6596,6596
AUTO THEFT,4627,4627,4627,4627,4627,4627


In [13]:
crimes_data = crimes_df2.loc[crimes_df2["Crime Count"] > 5000]
crimes_data

,Crime Count,Occurred Date,Location Type,Zip Code,Latitude,Longitude
Highest Offense Description,,,,,,
FAMILY DISTURBANCE,12449,12449,12449,12449,12449,12449
BURGLARY OF VEHICLE,10667,10667,10667,10667,10667,10667
THEFT,9395,9395,9395,9395,9395,9395
CRIMINAL MISCHIEF,6596,6596,6596,6596,6596,6596


In [14]:
len(crimes_data.index)

4

In [15]:
crimes_count = crimes_data["Crime Count"]
crimes_count

Highest Offense Description
FAMILY DISTURBANCE     12449
BURGLARY OF VEHICLE    10667
THEFT                   9395
CRIMINAL MISCHIEF       6596
Name: Crime Count, dtype: int64

In [21]:
final_covid_data = crimes_count.index
final_covid_data

Index(['FAMILY DISTURBANCE', 'BURGLARY OF VEHICLE', 'THEFT',
       'CRIMINAL MISCHIEF'],
      dtype='object', name='Highest Offense Description')

In [20]:
small_crime = covid_crime.loc[["FAMILY DISTURBABCE"], ["BURGLARY "]]

KeyError: "None of [Index(['FAMILY DISTURBABCE'], dtype='object')] are in the [index]"

In [22]:
covid_df = covid_crime.loc[covid_crime['Highest Offense Description'].isin(final_covid_data)]
covid_df.head()

,Highest Offense Description,Family Violence,Occurred Date,Location Type,Zip Code,Latitude,Longitude
34995,FAMILY DISTURBANCE,N,2021-06-20,HWY / ROAD / ALLEY/ STREET/ SIDEWALK,78745,30.198187,-97.769155
36016,BURGLARY OF VEHICLE,N,2020-04-03,PARKING /DROP LOT/ GARAGE,78759,30.412966,-97.719997
36026,THEFT,N,2021-10-14,OTHER / UNKNOWN,78613,30.477342,-97.802982
36245,THEFT,N,2021-10-14,SPECIALTY STORE,78759,30.390310,-97.740414
36739,CRIMINAL MISCHIEF,N,2021-03-16,PARK / PLAYGROUND,78724,30.324161,-97.677122


In [23]:
len(covid_df)

39107

In [24]:
covid_df.to_csv("covid_crime.csv", index = True)